You are provided with historical sales data for 1,115 Rossmann stores. The task is to forecast the "Sales" column for the test set. Note that some stores in the dataset were temporarily closed for refurbishment.

Files
train.csv - historical data including Sales

test.csv - historical data excluding Sales

sample_submission.csv - a sample submission file in the correct format

store.csv - supplemental information about the stores

Data fields

Most of the fields are self-explanatory. The following are descriptions for those that aren't.



Id - an Id that represents a (Store, Date) duple within the test set

Store - a unique Id for each store

Sales - the turnover for any given day (this is what you are predicting)

Customers - the number of customers on a given day

Open - an indicator for whether the store was open: 0 = closed, 1 = open

StateHoliday - indicates a state holiday. Normally all stores, with few exceptions, are closed on state holidays. Note that all schools are closed on public holidays and weekends. a = public holiday, b = Easter holiday, c = Christmas, 0 = None

SchoolHoliday - indicates if the (Store, Date) was affected by the closure of public schools

StoreType - differentiates between 4 different store models: a, b, c, d

Assortment - describes an assortment level: a = basic, b = extra, c = extended

CompetitionDistance - distance in meters to the nearest competitor store

CompetitionOpenSince[Month/Year] - gives the approximate year and month of the time the nearest competitor was opened

Promo - indicates whether a store is running a promo on that day

Promo2 - Promo2 is a continuing and consecutive promotion for some stores: 0 = store is not participating, 1 = store 
is participating

Promo2Since[Year/Week] - describes the year and calendar week when the store started participating in Promo2

PromoInterval - describes the consecutive intervals Promo2 is started, naming the months the promotion is started anew. E.g. "Feb,May,Aug,Nov" means each round starts in February, May, August, November of any given year for that store

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn import cross_validation


%matplotlib inline  


In [54]:
#read_data
train_data=pd.read_csv("dataset/train.csv")
store_data=pd.read_csv("dataset/store.csv")
test_data=pd.read_csv("dataset/test.csv")


/Users/jasdeep/kaggle/kaggle_env/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [55]:
#dimensions of data
print("There are {} examples and {} features in training data".format(train_data.shape[0],train_data.shape[1]))
print("There are {} examples and {} features in store data".format(store_data.shape[0],store_data.shape[1]))
print("There are {} examples and {} features in test data".format(test_data.shape[0],test_data.shape[1]))

There are 1017209 examples and 9 features in training data
There are 1115 examples and 10 features in store data
There are 41088 examples and 8 features in test data


In [56]:
#Sampling dataset
train_data.sample()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
870293,264,7,2013-05-12,0,0,0,0,0,0


In [57]:
store_data.sample()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
716,717,d,c,310.0,NaN,NaN,1,40.0,2011.0,"Jan,Apr,Jul,Oct"


In [58]:
test_data.sample()


,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
10692,10693,545,6,2015-09-05,1.0,0,0,0


In [59]:
percent_missing_train=train_data.isnull().sum()/train_data.isnull().count()
percent_missing_train

Store            0.0
DayOfWeek        0.0
Date             0.0
Sales            0.0
Customers        0.0
Open             0.0
Promo            0.0
StateHoliday     0.0
SchoolHoliday    0.0
dtype: float64

In [60]:
percent_missing_store=store_data.isnull().sum()/store_data.isnull().count()
percent_missing_store


Store                        0.000000
StoreType                    0.000000
Assortment                   0.000000
CompetitionDistance          0.002691
CompetitionOpenSinceMonth    0.317489
CompetitionOpenSinceYear     0.317489
Promo2                       0.000000
Promo2SinceWeek              0.487892
Promo2SinceYear              0.487892
PromoInterval                0.487892
dtype: float64

In [61]:
percent_missing_test=test_data.isnull().sum()/test_data.isnull().count()
percent_missing_test


Id               0.000000
Store            0.000000
DayOfWeek        0.000000
Date             0.000000
Open             0.000268
Promo            0.000000
StateHoliday     0.000000
SchoolHoliday    0.000000
dtype: float64

In [62]:
store_data[store_data[["Promo2SinceWeek","Promo2"]]["Promo2SinceWeek"].isnull()]

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
5,6,a,a,310.0,12.0,2013.0,0,NaN,NaN,NaN
6,7,a,c,24000.0,4.0,2013.0,0,NaN,NaN,NaN
7,8,a,a,7520.0,10.0,2014.0,0,NaN,NaN,NaN
8,9,a,c,2030.0,8.0,2000.0,0,NaN,NaN,NaN
9,10,a,a,3160.0,9.0,2009.0,0,NaN,NaN,NaN
15,16,a,c,3270.0,NaN,NaN,0,NaN,NaN,NaN
22,23,d,a,4060.0,8.0,2005.0,0,NaN,NaN,NaN


In [63]:
#making store data integral(storetype,Assortment)
store_data.loc[store_data['StoreType'] == 'a', 'StoreType'] = '1'
store_data.loc[store_data['StoreType'] == 'b', 'StoreType'] = '2'
store_data.loc[store_data['StoreType'] == 'c', 'StoreType'] = '3'
store_data.loc[store_data['StoreType'] == 'd', 'StoreType'] = '4'
store_data.loc[store_data['Assortment'] == 'a', 'Assortment'] = '1'
store_data.loc[store_data['Assortment'] == 'b', 'Assortment'] = '2'
store_data.loc[store_data['Assortment'] == 'c', 'Assortment'] = '3'
store_data

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,1,1,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,1,1,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,3,3,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,1,1,29910.0,4.0,2015.0,0,NaN,NaN,NaN
5,6,1,1,310.0,12.0,2013.0,0,NaN,NaN,NaN
6,7,1,3,24000.0,4.0,2013.0,0,NaN,NaN,NaN
7,8,1,1,7520.0,10.0,2014.0,0,NaN,NaN,NaN
8,9,1,3,2030.0,8.0,2000.0,0,NaN,NaN,NaN
9,10,1,1,3160.0,9.0,2009.0,0,NaN,NaN,NaN


In [64]:
#store open if nan
test_data.fillna(1, inplace=True)
#only train with open stores
train_data = train_data[train_data["Open"] != 0]

#combining store table
train_data = pd.merge(train_data, store_data, on='Store')
test_data = pd.merge(test_data, store_data, on='Store')


In [65]:
#converting dates into year, month,date
train_data['year'] = train_data.Date.apply(lambda x: x.split('-')[0])
train_data['year'] = train_data['year'].astype(float)
train_data['month'] = train_data.Date.apply(lambda x: x.split('-')[1])
train_data['month'] = train_data['month'].astype(float)
train_data['day'] = train_data.Date.apply(lambda x: x.split('-')[2])
train_data['day'] = train_data['day'].astype(float)
test_data['year'] = test_data.Date.apply(lambda x: x.split('-')[0])
test_data['year'] = test_data['year'].astype(float)
test_data['month'] = test_data.Date.apply(lambda x: x.split('-')[1])
test_data['month'] = test_data['month'].astype(float)
test_data['day'] = test_data.Date.apply(lambda x: x.split('-')[2])
test_data['day'] = test_data['day'].astype(float)
train_data

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,year,month,day
0,1,5,2015-07-31,5263,555,1,1,0,1,3,...,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,7.0,31.0
1,1,4,2015-07-30,5020,546,1,1,0,1,3,...,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,7.0,30.0
2,1,3,2015-07-29,4782,523,1,1,0,1,3,...,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,7.0,29.0
3,1,2,2015-07-28,5011,560,1,1,0,1,3,...,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,7.0,28.0
4,1,1,2015-07-27,6102,612,1,1,0,1,3,...,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,7.0,27.0
5,1,6,2015-07-25,4364,500,1,0,0,0,3,...,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,7.0,25.0
6,1,5,2015-07-24,3706,459,1,0,0,0,3,...,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,7.0,24.0
7,1,4,2015-07-23,3769,503,1,0,0,0,3,...,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,7.0,23.0
8,1,3,2015-07-22,3464,463,1,0,0,0,3,...,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,7.0,22.0
9,1,2,2015-07-21,3558,469,1,0,0,0,3,...,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,7.0,21.0


In [66]:
test_data

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,year,month,day
0,1,1,4,2015-09-17,1.0,1,0,0,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,9.0,17.0
1,857,1,3,2015-09-16,1.0,1,0,0,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,9.0,16.0
2,1713,1,2,2015-09-15,1.0,1,0,0,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,9.0,15.0
3,2569,1,1,2015-09-14,1.0,1,0,0,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,9.0,14.0
4,3425,1,7,2015-09-13,0.0,0,0,0,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,9.0,13.0
5,4281,1,6,2015-09-12,1.0,0,0,0,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,9.0,12.0
6,5137,1,5,2015-09-11,1.0,0,0,0,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,9.0,11.0
7,5993,1,4,2015-09-10,1.0,0,0,0,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,9.0,10.0
8,6849,1,3,2015-09-09,1.0,0,0,0,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,9.0,9.0
9,7705,1,2,2015-09-08,1.0,0,0,0,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN,2015.0,9.0,8.0


In [67]:
#removing nans
test_data.fillna(0, inplace=True)
train_data.fillna(0, inplace=True)

train_data

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,year,month,day
0,1,5,2015-07-31,5263,555,1,1,0,1,3,...,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,7.0,31.0
1,1,4,2015-07-30,5020,546,1,1,0,1,3,...,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,7.0,30.0
2,1,3,2015-07-29,4782,523,1,1,0,1,3,...,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,7.0,29.0
3,1,2,2015-07-28,5011,560,1,1,0,1,3,...,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,7.0,28.0
4,1,1,2015-07-27,6102,612,1,1,0,1,3,...,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,7.0,27.0
5,1,6,2015-07-25,4364,500,1,0,0,0,3,...,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,7.0,25.0
6,1,5,2015-07-24,3706,459,1,0,0,0,3,...,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,7.0,24.0
7,1,4,2015-07-23,3769,503,1,0,0,0,3,...,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,7.0,23.0
8,1,3,2015-07-22,3464,463,1,0,0,0,3,...,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,7.0,22.0
9,1,2,2015-07-21,3558,469,1,0,0,0,3,...,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,7.0,21.0


In [68]:
test_data

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,year,month,day
0,1,1,4,2015-09-17,1.0,1,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,9.0,17.0
1,857,1,3,2015-09-16,1.0,1,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,9.0,16.0
2,1713,1,2,2015-09-15,1.0,1,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,9.0,15.0
3,2569,1,1,2015-09-14,1.0,1,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,9.0,14.0
4,3425,1,7,2015-09-13,0.0,0,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,9.0,13.0
5,4281,1,6,2015-09-12,1.0,0,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,9.0,12.0
6,5137,1,5,2015-09-11,1.0,0,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,9.0,11.0
7,5993,1,4,2015-09-10,1.0,0,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,9.0,10.0
8,6849,1,3,2015-09-09,1.0,0,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,9.0,9.0
9,7705,1,2,2015-09-08,1.0,0,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,2015.0,9.0,8.0


In [69]:
#deleting id's
del test_data['Id']
del test_data['Date']
del test_data['PromoInterval']
del train_data['StateHoliday']


del train_data['Customers']
del train_data['Date']
del train_data['PromoInterval']
del test_data['StateHoliday']


train_data['StoreType'] = train_data['StoreType'].astype(float)
test_data['StoreType'] = test_data['StoreType'].astype(float)

train_data['Assortment'] = train_data['Assortment'].astype(float)
test_data['Assortment'] = test_data['Assortment'].astype(float)









In [77]:
train_features=list(test_data.columns)
train_features

['Store',
 'DayOfWeek',
 'Open',
 'Promo',
 'SchoolHoliday',
 'StoreType',
 'Assortment',
 'CompetitionDistance',
 'CompetitionOpenSinceMonth',
 'CompetitionOpenSinceYear',
 'Promo2',
 'Promo2SinceWeek',
 'Promo2SinceYear',
 'year',
 'month',
 'day']

In [78]:
test_data.columns

Index(['Store', 'DayOfWeek', 'Open', 'Promo', 'SchoolHoliday', 'StoreType',
       'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
       'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek',
       'Promo2SinceYear', 'year', 'month', 'day'],
      dtype='object')

In [83]:
acc=0.9
def ToWeight(y):
    w = np.zeros(y.shape, dtype=float)
    ind = y != 0
    w[ind] = 1./(y[ind]**2)
    return w


def rmspe(yhat, y):
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean( w * (y - yhat)**2 ))
    return rmspe


def rmspe_xg(yhat, y):
    # y = y.values
    y = y.get_label()
    y = np.exp(y) - 1
    yhat = np.exp(yhat) - 1
    w = ToWeight(y)
    rmspe = np.sqrt(np.mean(w * (y - yhat)**2))
    return "rmspe", rmspe

In [82]:
params = {"objective": "reg:linear",
          "eta": 0.3,
          "max_depth": 8,
          "subsample": 0.7,
          "colsample_bytree": 0.7,
          "silent": 1
          }
num_trees = 300

print("Train a XGBoost model")
val_size = 100000
X_train, X_test = cross_validation.train_test_split(train_data, test_size=0.01)
dtrain = xgb.DMatrix(X_train[train_features], np.log(X_train["Sales"] + 1))
dvalid = xgb.DMatrix(X_test[train_features], np.log(X_test["Sales"] + 1))
dtest = xgb.DMatrix(test_data)
watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
gbm = xgb.train(params, dtrain, num_trees, evals=watchlist, early_stopping_rounds=50, feval=rmspe_xg, verbose_eval=True)

print("Validating")
train_probs = gbm.predict(xgb.DMatrix(X_test[train_features]))
indices = train_probs < 0
train_probs[indices] = 0
error = rmspe(np.exp(train_probs) - 1, X_test['Sales'].values)
print('error', error)

print("Make predictions on the test set")
new_test_read=pd.read_csv("dataset/test.csv")
test_probs = gbm.predict(xgb.DMatrix(test_data[train_features]))
indices = test_probs < 0
test_probs[indices] = 0
submission = pd.DataFrame({"Id": new_test_read["Id"], "Sales": np.exp(test_probs) - 1})
submission.to_csv("xgboost_kscript_submission.csv", index=False)

Train a XGBoost model
[0]	eval-rmse:5.79554	train-rmse:5.79474	eval-rmspe:0.996707	train-rmspe:0.996787
Multiple eval metrics have been passed: 'train-rmspe' will be used for early stopping.

Will train until train-rmspe hasn't improved in 50 rounds.
[1]	eval-rmse:4.0658	train-rmse:4.06444	eval-rmspe:0.98133	train-rmspe:0.981384
[2]	eval-rmse:2.85858	train-rmse:2.85622	eval-rmspe:0.937659	train-rmspe:0.937663
[3]	eval-rmse:2.0164	train-rmse:2.01316	eval-rmspe:0.856074	train-rmspe:0.856157
[4]	eval-rmse:1.433	train-rmse:1.42803	eval-rmspe:0.743009	train-rmspe:0.743553
[5]	eval-rmse:1.03023	train-rmse:1.02417	eval-rmspe:0.617623	train-rmspe:0.61986
[6]	eval-rmse:0.758169	train-rmse:0.750173	eval-rmspe:0.501286	train-rmspe:0.506773
[7]	eval-rmse:0.577006	train-rmse:0.566328	eval-rmspe:0.407888	train-rmspe:0.418154
[8]	eval-rmse:0.460755	train-rmse:0.446443	eval-rmspe:0.342699	train-rmspe:0.358282
[9]	eval-rmse:0.392024	train-rmse:0.374992	eval-rmspe:0.307072	train-rmspe:0.32722
[10]	eval-

[96]	eval-rmse:0.175524	train-rmse:0.13883	eval-rmspe:0.13211	train-rmspe:0.18716
[97]	eval-rmse:0.175362	train-rmse:0.138605	eval-rmspe:0.131779	train-rmspe:0.186995
[98]	eval-rmse:0.175258	train-rmse:0.13849	eval-rmspe:0.13162	train-rmspe:0.186868
[99]	eval-rmse:0.174975	train-rmse:0.138197	eval-rmspe:0.131117	train-rmspe:0.186584
[100]	eval-rmse:0.174689	train-rmse:0.137784	eval-rmspe:0.130702	train-rmspe:0.186288
[101]	eval-rmse:0.174424	train-rmse:0.137351	eval-rmspe:0.130362	train-rmspe:0.185999
[102]	eval-rmse:0.174241	train-rmse:0.136943	eval-rmspe:0.130132	train-rmspe:0.185334
[103]	eval-rmse:0.174122	train-rmse:0.136755	eval-rmspe:0.129969	train-rmspe:0.185083
[104]	eval-rmse:0.174158	train-rmse:0.136325	eval-rmspe:0.12967	train-rmspe:0.184846
[105]	eval-rmse:0.174005	train-rmse:0.136149	eval-rmspe:0.129538	train-rmspe:0.184643
[106]	eval-rmse:0.173887	train-rmse:0.135925	eval-rmspe:0.129301	train-rmspe:0.18458
[107]	eval-rmse:0.172984	train-rmse:0.135725	eval-rmspe:0.129179	

[192]	eval-rmse:0.163096	train-rmse:0.119282	eval-rmspe:0.113061	train-rmspe:0.154147
[193]	eval-rmse:0.162928	train-rmse:0.119007	eval-rmspe:0.112561	train-rmspe:0.153794
[194]	eval-rmse:0.162861	train-rmse:0.118872	eval-rmspe:0.112405	train-rmspe:0.153609
[195]	eval-rmse:0.162839	train-rmse:0.118784	eval-rmspe:0.112337	train-rmspe:0.153664
[196]	eval-rmse:0.161652	train-rmse:0.118682	eval-rmspe:0.112107	train-rmspe:0.153601
[197]	eval-rmse:0.161539	train-rmse:0.118641	eval-rmspe:0.112084	train-rmspe:0.153544
[198]	eval-rmse:0.161487	train-rmse:0.11859	eval-rmspe:0.112031	train-rmspe:0.153334
[199]	eval-rmse:0.161462	train-rmse:0.11854	eval-rmspe:0.112003	train-rmspe:0.153315
[200]	eval-rmse:0.161404	train-rmse:0.118373	eval-rmspe:0.111915	train-rmspe:0.153206
[201]	eval-rmse:0.162264	train-rmse:0.118208	eval-rmspe:0.111777	train-rmspe:0.15307
[202]	eval-rmse:0.161918	train-rmse:0.118094	eval-rmspe:0.111661	train-rmspe:0.152955
[203]	eval-rmse:0.161872	train-rmse:0.117963	eval-rmspe:0

[288]	eval-rmse:0.153484	train-rmse:0.11062	eval-rmspe:0.106106	train-rmspe:0.145465
[289]	eval-rmse:0.151363	train-rmse:0.11049	eval-rmspe:0.106141	train-rmspe:0.14537
[290]	eval-rmse:0.151407	train-rmse:0.110439	eval-rmspe:0.106108	train-rmspe:0.145164
[291]	eval-rmse:0.151306	train-rmse:0.110343	eval-rmspe:0.106055	train-rmspe:0.14513
[292]	eval-rmse:0.151171	train-rmse:0.110297	eval-rmspe:0.106029	train-rmspe:0.144936
[293]	eval-rmse:0.151131	train-rmse:0.110241	eval-rmspe:0.105975	train-rmspe:0.144915
[294]	eval-rmse:0.151139	train-rmse:0.110153	eval-rmspe:0.105899	train-rmspe:0.144859
[295]	eval-rmse:0.151094	train-rmse:0.110099	eval-rmspe:0.105842	train-rmspe:0.14482
[296]	eval-rmse:0.150995	train-rmse:0.110031	eval-rmspe:0.105848	train-rmspe:0.144697
[297]	eval-rmse:0.150392	train-rmse:0.109994	eval-rmspe:0.10579	train-rmspe:0.14465
[298]	eval-rmse:0.150391	train-rmse:0.109939	eval-rmspe:0.105778	train-rmspe:0.144553
[299]	eval-rmse:0.150221	train-rmse:0.109826	eval-rmspe:0.105

In [84]:
print("The accuracy of model is {}".format(acc))

The accuracy of model is 0.9
